In [2]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import pdfplumber

pdf_path = 'C:/Users/SSTLabs/Desktop/여형구/설명서.pdf'

# PDF 파일을 열고 내용을 읽습니다.
with pdfplumber.open(pdf_path) as pdf:
    total_text = ''
    # 모든 페이지를 순회하며 텍스트를 추출합니다.
    for page in pdf.pages:
        total_text += page.extract_text()

print(total_text)


ImportError: cannot import name 'COMMON_SAFE_ASCII_CHARACTERS' from 'charset_normalizer.constant' (c:\Users\SSTLabs\miniconda3\envs\wahoman\lib\site-packages\charset_normalizer\constant.py)